# Avaliação de Agentes

Temos um assistente de email que usa um roteador para triagem de emails e então passa o email para o agente gerar uma resposta. Como podemos ter certeza de que funcionará bem em produção? É por isso que os testes são importantes: eles orientam nossas decisões sobre a arquitetura do agente com métricas quantificáveis como qualidade da resposta, uso de tokens, latência ou precisão da triagem. O [LangSmith](https://docs.smith.langchain.com/) oferece duas maneiras principais de testar agentes.

![overview-img](img/overview_eval.png)

#### Carregar Variáveis de Ambiente

In [ ]:
from dotenv import load_dotenv
load_dotenv("../.env")

## Como Executar Avaliações

#### Pytest / Vitest

[Pytest](https://docs.pytest.org/en/stable/) e Vitest são bem conhecidos por muitos desenvolvedores como ferramentas poderosas para escrever testes dentro dos ecossistemas Python e JavaScript. O LangSmith se integra com esses frameworks para permitir que você escreva e execute testes que registram resultados no LangSmith. Para este notebook, usaremos Pytest.
* Pytest é uma ótima maneira de começar para desenvolvedores que já estão familiarizados com o framework.
* Pytest é ótimo para avaliações mais complexas, onde cada caso de teste do agente requer verificações específicas e critérios de sucesso que são mais difíceis de generalizar.

#### Datasets do LangSmith

Você também pode criar um dataset [no LangSmith](https://docs.smith.langchain.com/evaluation) e executar nosso assistente contra o dataset usando a API de avaliação do LangSmith.
* Os datasets do LangSmith são ótimos para equipes que estão construindo colaborativamente sua suíte de testes.
* Você pode aproveitar traces de produção, filas de anotação, geração de dados sintéticos, e mais, para adicionar exemplos a um dataset dourado sempre crescente.
* Os datasets do LangSmith são ótimos quando você pode definir avaliadores que podem ser aplicados a todos os casos de teste no dataset (ex. similaridade, precisão de correspondência exata, etc.)

## Casos de Teste

Os testes geralmente começam com a definição dos casos de teste, o que pode ser um processo desafiador. Neste caso, vamos apenas definir um conjunto de emails de exemplo que queremos gerenciar junto com algumas coisas para testar. Você pode ver os casos de teste em `eval/email_dataset.py`, que contém o seguinte:

1. **Emails de Entrada**: Uma coleção de exemplos diversos de email
2. **Classificações Verdadeiras**: `Respond`, `Notify`, `Ignore`
3. **Chamadas de Ferramentas Esperadas**: Ferramentas chamadas para cada email que requer uma resposta
4. **Critérios de Resposta**: O que torna uma boa resposta para emails que requerem respostas

Note que temos tanto:
- Testes "de integração" ponta a ponta (ex. Emails de Entrada -> Agente -> Saída Final vs Critérios de Resposta)
- Testes para etapas específicas em nosso fluxo de trabalho (ex. Emails de Entrada -> Agente -> Classificação vs Classificação Verdadeira)

In [ ]:

%load_ext autoreload
%autoreload 2

from email_assistant.eval.email_dataset import email_inputs, expected_tool_calls, triage_outputs_list, response_criteria_list

test_case_ix = 0

print("Entrada de Email:", email_inputs[test_case_ix])
print("Saída de Triagem Esperada:", triage_outputs_list[test_case_ix])
print("Chamadas de Ferramenta Esperadas:", expected_tool_calls[test_case_ix])
print("Critérios de Resposta:", response_criteria_list[test_case_ix])

## Exemplo com Pytest

Vamos ver como podemos escrever um teste para uma parte específica do nosso fluxo de trabalho com Pytest. Testaremos se nosso `email_assistant` faz as chamadas de ferramentas corretas ao responder aos emails.

In [ ]:
import pytest
from email_assistant.eval.email_dataset import email_inputs, expected_tool_calls
from email_assistant.utils import format_messages_string
from email_assistant.email_assistant import email_assistant
from email_assistant.utils import extract_tool_calls

from langsmith import testing as t

@pytest.mark.langsmith
@pytest.mark.parametrize(
    "email_input, expected_calls",
    [   # Pick some examples with e-mail reply expected
        (email_inputs[0],expected_tool_calls[0]),
        (email_inputs[3],expected_tool_calls[3]),
    ],
)
def test_email_dataset_tool_calls(email_input, expected_calls):
    """Test if email processing contains expected tool calls.

    This test confirms that all expected tools are called during email processing,
    but does not check the order of tool invocations or the number of invocations
    per tool. Additional checks for these aspects could be added if desired.
    """
    # Run the email assistant
    messages = [{"role": "user", "content": str(email_input)}]
    result = email_assistant.invoke({"messages": messages})

    # Extract tool calls from messages list
    extracted_tool_calls = extract_tool_calls(result['messages'])

    # Check if all expected tool calls are in the extracted ones
    missing_calls = [call for call in expected_calls if call.lower() not in extracted_tool_calls]

    t.log_outputs({
                "missing_calls": missing_calls,
                "extracted_tool_calls": extracted_tool_calls,
                "response": format_messages_string(result['messages'])
            })

    # Test passes if no expected calls are missing
    assert len(missing_calls) == 0

Você notará algumas coisas:
- Para [executar com Pytest e registrar resultados de teste no LangSmith](https://docs.smith.langchain.com/evaluation/how_to_guides/pytest), precisamos apenas adicionar o decorador `@pytest.mark.langsmith` à nossa função e colocá-la em um arquivo, como você vê em `notebooks/test_tools.py`. Isso registrará os resultados do teste no LangSmith.
- Segundo, podemos passar exemplos de dataset para a função de teste como mostrado [aqui](https://docs.smith.langchain.com/evaluation/how_to_guides/pytest#parametrize-with-pytestmarkparametrize) via `@pytest.mark.parametrize`.

#### Executando Pytest
Podemos executar o teste da linha de comando. Definimos o código acima em um arquivo python. Da raiz do projeto, execute:

`! LANGSMITH_TEST_SUITE='Email assistant: Test Tools For Interrupt'  pytest notebooks/test_tools.py`

#### Visualizando Resultado do Experimento

Podemos visualizar os resultados na interface do LangSmith. O `assert len(missing_calls) == 0` é registrado na coluna `Pass` no LangSmith. Os `log_outputs` são passados para a coluna `Outputs` e os argumentos da função são passados para a coluna `Inputs`. Cada entrada passada em `@pytest.mark.parametrize(` é uma linha separada registrada no nome do projeto `LANGSMITH_TEST_SUITE` no LangSmith, que é encontrado em `Datasets & Experiments`.

![Test Results](img/test_result.png)

## Exemplo de Datasets do LangSmith

![overview-img](img/eval_detail.png)

Vamos ver como podemos executar avaliações com datasets do LangSmith. No exemplo anterior com Pytest, avaliamos a precisão de chamada de ferramenta do assistente de email. Agora, o dataset que vamos avaliar aqui é especificamente para a etapa de triagem do assistente de email, classificando se um email requer uma resposta.

#### Definição do Dataset

Podemos [criar um dataset no LangSmith](https://docs.smith.langchain.com/evaluation/how_to_guides/manage_datasets_programmatically#create-a-dataset) com o SDK do LangSmith. O código abaixo cria um dataset com os casos de teste no arquivo `eval/email_dataset.py`.

In [ ]:
from langsmith import Client

from email_assistant.eval.email_dataset import examples_triage

# Initialize LangSmith client
client = Client()

# Dataset name
dataset_name = "E-mail Triage Evaluation"

# Create dataset if it doesn't exist
if not client.has_dataset(dataset_name=dataset_name):
    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description="A dataset of e-mails and their triage decisions."
    )
    # Add examples to the dataset
    client.create_examples(dataset_id=dataset.id, examples=examples_triage)

#### Função Alvo

O dataset tem a seguinte estrutura, com uma entrada de e-mail e uma classificação de triagem verdadeira para o e-mail como saída:

```
examples_triage = [
  {
      "inputs": {"email_input": email_input_1},
      "outputs": {"classification": triage_output_1},   # NOTA: Isso se torna reference_output no dataset criado
  }, ...
]
```

In [ ]:
print("Entrada de Exemplo do Dataset (inputs):", examples_triage[0]['inputs'])

In [ ]:
print("Saída de Referência de Exemplo do Dataset (reference_outputs):", examples_triage[0]['outputs'])

Definimos uma função que recebe as entradas do dataset e as passa para nosso assistente de email. A [API de avaliação](https://docs.smith.langchain.com/evaluation) do LangSmith passa o dicionário `inputs` para esta função. Esta função então retorna um dicionário com a saída do agente. Como estamos avaliando a etapa de triagem, precisamos apenas retornar a decisão de classificação.

In [ ]:
def target_email_assistant(inputs: dict) -> dict:
    """Process an email through the workflow-based email assistant."""
    response = email_assistant.nodes['triage_router'].invoke({"email_input": inputs["email_input"]})
    return {"classification_decision": response.update['classification_decision']}

#### Função Avaliadora

Agora, criamos uma função avaliadora. O que queremos avaliar? Temos saídas de referência em nosso dataset e saídas do agente definidas nas funções acima.

* Saídas de referência: `"reference_outputs": {"classification": triage_output_1} ...`
* Saídas do agente: `"outputs": {"classification_decision": agent_output_1} ...`

Queremos avaliar se a saída do agente corresponde à saída de referência. Então, simplesmente precisamos de uma função avaliadora que compare as duas, onde `outputs` é a saída do agente e `reference_outputs` é a saída de referência do dataset.

In [ ]:
def classification_evaluator(outputs: dict, reference_outputs: dict) -> bool:
    """Check if the answer exactly matches the expected answer."""
    return outputs["classification_decision"].lower() == reference_outputs["classification"].lower()

### Executando Avaliação

Agora, a pergunta é: como essas coisas se conectam? A API de avaliação cuida disso para nós. Ela passa o dicionário `inputs` do nosso dataset para a função alvo. Ela passa o dicionário `reference_outputs` do nosso dataset para a função avaliadora. E ela passa os `outputs` do nosso agente para a função avaliadora.

Note que isso é similar ao que fizemos com Pytest: no Pytest, passamos as entradas e saídas de referência do exemplo do dataset para a função de teste com `@pytest.mark.parametrize`.

In [ ]:
# Set to true if you want to kick off evaluation
run_expt = True
if run_expt:
    experiment_results_workflow = client.evaluate(
        # Run agent
        target_email_assistant,
        # Dataset name
        data=dataset_name,
        # Evaluator
        evaluators=[classification_evaluator],
        # Name of the experiment
        experiment_prefix="E-mail assistant workflow",
        # Number of concurrent evaluations
        max_concurrency=2,
    )

Podemos visualizar os resultados de ambos os experimentos na interface do LangSmith.

![Test Results](img/eval.png)

## Avaliação LLM-como-Juiz

Mostramos testes unitários para a etapa de triagem (usando evaluate()) e chamada de ferramenta (usando Pytest).

Vamos demonstrar como você poderia usar um LLM como juiz para avaliar a execução do nosso agente contra um conjunto de critérios de sucesso.

![types](img/eval_types.png)

Primeiro, definimos um schema de saída estruturada para nosso avaliador LLM que contém uma nota e justificativa para a nota.

In [ ]:
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model

class CriteriaGrade(BaseModel):
    """Pontuar a resposta contra critérios específicos."""
    justification: str = Field(description="A justificativa para a nota e pontuação, incluindo exemplos específicos da resposta.")
    grade: bool = Field(description="A resposta atende aos critérios fornecidos?")

# Criar um LLM global para avaliação para evitar recriar para cada teste
criteria_eval_llm = init_chat_model("gemini-2.5-flash", model_provider="google-genai")
criteria_eval_structured_llm = criteria_eval_llm.with_structured_output(CriteriaGrade)

In [ ]:
email_input = email_inputs[0]
print("Entrada de Email:", email_input)
success_criteria = response_criteria_list[0]
print("Critérios de Sucesso:", success_criteria)

Nosso Assistente de Email é invocado com a entrada de email e a resposta é formatada em uma string. Essas são então passadas para o avaliador LLM para receber uma nota e justificativa para a nota.

In [ ]:
response = email_assistant.invoke({"email_input": email_input})

In [ ]:
from email_assistant.eval.prompts import RESPONSE_CRITERIA_SYSTEM_PROMPT

all_messages_str = format_messages_string(response['messages'])
eval_result = criteria_eval_structured_llm.invoke([
        {"role": "system",
            "content": RESPONSE_CRITERIA_SYSTEM_PROMPT},
        {"role": "user",
            "content": f"""\n\n Response criteria: {success_criteria} \n\n Assistant's response: \n\n {all_messages_str} \n\n Evaluate whether the assistant's response meets the criteria and provide justification for your evaluation."""}
    ])

eval_result

In [ ]:
RESPONSE_CRITERIA_SYSTEM_PROMPT

Podemos ver que o avaliador LLM retorna um resultado de avaliação com um schema correspondente ao nosso modelo base `CriteriaGrade`.

## Executando contra uma Suíte de Testes Maior
Agora que vimos como avaliar nosso agente usando Pytest e evaluate(), e vimos um exemplo de usar um LLM como juiz, podemos usar avaliações sobre uma suíte de testes maior para ter uma melhor noção de como nosso agente se comporta em uma variedade mais ampla de exemplos.

Let's run our email_assistant against a larger test suite.
```
! LANGSMITH_TEST_SUITE='Email assistant: Test Full Response Interrupt' LANGSMITH_EXPERIMENT='email_assistant' pytest tests/test_response.py --agent-module email_assistant
```

In `test_response.py`, you can see a few things. 

We pass our dataset examples into functions that will run pytest and log to our `LANGSMITH_TEST_SUITE`:

```
# Reference output key
@pytest.mark.langsmith(output_keys=["criteria"])
# Variable names and a list of tuples with the test cases
# Each test case is (email_input, email_name, criteria, expected_calls)
@pytest.mark.parametrize("email_input,email_name,criteria,expected_calls",create_response_test_cases())
def test_response_criteria_evaluation(email_input, email_name, criteria, expected_calls):
```

We use LLM-as-judge with a grading schema:
```
class CriteriaGrade(BaseModel):
    """Score the response against specific criteria."""
    grade: bool = Field(description="Does the response meet the provided criteria?")
    justification: str = Field(description="The justification for the grade and score, including specific examples from the response.")
```


We evaluate the agent response relative to the criteria:
```
    # Evaluate against criteria
    eval_result = criteria_eval_structured_llm.invoke([
        {"role": "system",
            "content": RESPONSE_CRITERIA_SYSTEM_PROMPT},
        {"role": "user",
            "content": f"""\n\n Response criteria: {criteria} \n\n Assistant's response: \n\n {all_messages_str} \n\n Evaluate whether the assistant's response meets the criteria and provide justification for your evaluation."""}
    ])
```

Agora vamos dar uma olhada neste experimento na interface do LangSmith e ver em que nosso agente foi bom, e o que poderia melhorar.

#### Obtendo Resultados

Também podemos obter os resultados da avaliação lendo o projeto de rastreamento associado ao nosso experimento. Isso é ótimo para criar visualizações customizadas do desempenho do nosso agente.

In [ ]:
# TODO: Copy your experiment name here
experiment_name = "email_assistant:8286b3b8"
# Set this to load expt results
load_expt = False
if load_expt:
    email_assistant_experiment_results = client.read_project(project_name=experiment_name, include_stats=True)
    print("Latência p50:", email_assistant_experiment_results.latency_p50)
    print("Latência p99:", email_assistant_experiment_results.latency_p99)
    print("Uso de Tokens:", email_assistant_experiment_results.total_tokens)
    print("Estatísticas de Feedback:", email_assistant_experiment_results.feedback_stats)